In [ ]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init("/home/pyspark/spark")

In [ ]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .enableHiveSupport() \
    .getOrCreate()


# spark.xxxxxと記載することで処理を分散させることが可能です。

# 今回利用するデータの確認

今回は、datafileフォルダ配下の

- jinko.csv(各年代の都道府県ごとの人口)
- kenmei_master.csv(都道府県コードをまとめている)

を利用していきます。

In [ ]:
#　データの読み込みを行う

from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col
import pyspark.sql.functions as F

struct = StructType([
    StructField("code", StringType(), False),
    StructField("gengo", StringType(), False),
    StructField("wareki", StringType(), False),
    StructField("seireki", StringType(), False),
    StructField("chu", StringType(), False),
    StructField("sokei", StringType(), False),
    StructField("jinko_male", StringType(), False),
    StructField("jinko_female", StringType(), False)
])
df=spark.read.option("multiLine", "true").option("encoding", "UTF-8") \
    .csv("./datafile/jinko.csv", header=False, sep=',', inferSchema=False,schema=struct)

struct2 = StructType([
    StructField("code", StringType(), False),
    StructField("kenmei", StringType(), False)
])
df2=spark.read.option("multiLine", "true").option("encoding", "UTF-8") \
    .csv("./datafile/kenmei_master.csv", header=False, sep=',', inferSchema=False,schema=struct2)


struct3 = StructType([
    StructField("codes", StringType(), False),
    StructField("kenmei", StringType(), False)
])
df3=spark.read.option("multiLine", "true").option("encoding", "UTF-8") \
    .csv("./datafile/kenmei_master.csv", header=False, sep=',', inferSchema=False,schema=struct3)


In [ ]:
# それぞれのデータの確認をしてみましょう
df.show()

In [ ]:
df2.show()

In [ ]:
df3.show()

# テストレベルの設定

どの単位でテストを行うかを考えることをテストレベルを設定すると言います。

- テーブル単位でのテスト
- カラム単位でのテスト
- テーブル間単位でのテスト

の3種類存在します。

例えば、1つ〜カラムに対して確認を行うのであれば、カラム単位のテスト(辞書テスト、if-thenテストなど)です。  
テーブル単位でのテストは一つのテーブル単位でテストを行うことです（0件テストやタイムラインネスなど）  
テーブル間単位でのテストは、複数テーブル間でのテストを行うことです（コンシステンシーなど）  


# まずは自身でデータを理解して、定義を考えていきます。
  

# 現状把握のための便利関数
最大（maximum）、最小（ minimum）、平均（ mean）、中央値（ median）、最頻値（mode）、分散（ variance）、標準偏差（ standard deviation）基本統計量を取得を紹介します。

主にデータの傾向を掴むために利用されます。

テストというより、現状把握の意味合いの方が高いかもしれません。

In [ ]:
# テストとして役に立つことはあまり多くありませんが、データをさっと確認する時に役に立ちます
# 確認後データのテスト計画を立てていくことになります。
df.summary().show()

# If-thenテスト
もしAの値が1ならばBの値は2のようなテストを行うのがif-thenテストです。

In [ ]:
# if-then
# カラム
df = df.withColumn("gengo_wareki_if_then_check",
    F.when(F.col("gengo") == "大正", 
        ((F.col("wareki").cast("integer") > 0) & (F.col("wareki").cast("integer") <= 62)).cast("long")
    )
)

# ゼロコントロール
四則演算の結果について確認するのが、ゼロコントロールです。

In [ ]:
# ゼロコントロール
# カラム
df.withColumn(
    'sokei_check_zero_control', 
    (F.col('sokei') == (F.col('jinko_male') + F.col('jinko_female'))).cast("long")
).show()

# レンジテストと辞書テスト
データが特定の範囲に入っているのか？確認するのがレンジテストと辞書テストです。

In [ ]:
# 辞書テスト
# カラム
df.withColumn("gengo_dictionary_chek", F.col("gengo").isin(['大正','昭和','平成'])).show()

# レンジテスト
# カラム
df.orderBy(F.col("seireki").desc()).show()
df.orderBy(F.col("seireki").asc()).show()

df.withColumn("seireki_range_check", (F.col("seireki").between(1920,2015)).cast("long")).show()
df.withColumn("seireki_range_check", F.col("seireki").between(1920,2015)).groupby("seireki_range_check").count().show()

#この時点ででた変なデータは後ほどリペアのレクチャーで除外します。

# Nullチェックとユニークネス
データにNullが含まれていたりユニークでないとデータが非常に扱いにくいです。

Nullチェックとユニークネスを通して扱いにくいデータを見つけ出していきましょう。

In [ ]:
# ユニークネス、PK
#　こちらはPKの確認
df.select(F.countDistinct("code","gengo", "wareki")).show()

# 割合を判定する
# ユニークネス
#全体に対してどれだけユニークか？
#countdistinct / 全体のレコード
df.withColumn("unique_ness_check", 
    F.lit(df.agg(F.countDistinct("code","gengo", "wareki").alias("countdistinct")).collect()[0][0]) / F.lit(df.count())).show()

# ユニークではない！

# どうもnullがある様子
>>> df.groupby("kenmei").count().show(n=60)


# パターンチェック
特定の正規表現にデータが沿っているのか？をチェックするパターンチェックについて学びます

In [ ]:
# 正規表現はJavaの正規表現き表です
# パターンチェック
# カラム
df.withColumn("seireki_pattern_chek", (F.col("seireki").rlike("\d{4}")).cast("long")).show()

# コンシステンシー
テーブル間の紐付きの割合で見るのは、エクスターナルコンシステンシー

エクスターナルコンシステンシー  
joinできるの？というのは大きな問題である  
データは複数組み合わせて価値を生み出すものなので単体では役に立たないことが多い

In [ ]:
# 今回はdfのcodeとdf2のcodeがどれだけ紐つくかを確認する
# テーブル間

# A-Bをして残ったら一致していないものがある
df.select("code").subtract(df2.select("code")).show()

# 全体の件数と、一致数件数の割合をとってみる方式でもOK
df.withColumn("code_consistency_check", 
    F.lit(df.select("code").distinct().count()) / F.lit(df.select("code").intersect(df2.select("code")).count())).show()

# レイショーコントロール

想定した割合にデータの件数や統計量が収まっているかどうかをテストする方法。割合制御とも呼ばれる。  
男女の出生率がおおよそ1：1であることを利用して集めたデータの男女比に、極端な差がないかの比を比較し確認することなども含まれる

また、急にデータが増えたなどのチェックにも使われる。    
例えばリリースの不具合で急激に件数が増えた！  
なんてことにも利用できたりします。

In [ ]:
# ratio_control
# カラム(テーブルでも可能。例えばテーブルの件数など)
df.withColumn("jinko_male_jinko_female_ratio_check", 
    F.col("jinko_male").cast("integer") / F.col("jinko_female").cast("integer")) \
        .withColumn("ratio_check", (F.col("jinko_male_jinko_female_ratio_check").between(0.8, 1.2)).cast("long")).show()

# タイムラインネス
データがしっかりと特定の時間に処理されているか確認する方法です。

In [ ]:
# タイムラインネス
# カラム
# 少し運用ちっくですが、必ずETLなどで処理した時間をテーブルの末尾に追加しておくと良いです。
# pythonであればosの機能を使ってファイルの更新時間を取得することができますが、分散基盤になると使いづらいのです。
df.withColumn("timelineness_check", F.current_timestamp()).show()

# メタデータの品質テスト

メタデータの名寄せ  
code で　一方が idだったらjoinをためらってしまいませんか？  
事前に準備するというより、既にめちゃくちゃな状態でそれを修正するために探索していくことが多いです。  
そのため、データのフォーマットから実は同じじゃない？というサジェストをしていくと良い  


今回はdf2とdf3のチグハグについて考えてみようと思います  
みると一目瞭然ですが、一方はcodes、もう一方はcodeになっています  
PJとして2桁の数値はcodeという名称とした場合  
そんな時に使えるのがエクスターナルコンシステンシーです.


つまり一致数が高ければ、「あれ？これって同じ定義じゃないですか？」と言ったサジェストができることになります。

In [ ]:
# メタデータのサジェスト

# コンシステンシーを確認
# df2 <-> df3
hoge = df.withColumn("master_data_consistency_check", 
    F.lit(df2.select("code").distinct().count()) / F.lit(df3.select("codes").intersect(df2.select("code")).count()))

hoge.withColumn("code_metadata_suggest", F.when(F.col("master_data_consistency_check").cast("integer") > 0.8, True))

# 0件チェック
テーブル単位でのテスト。  
急にデータが更新されていなかったりする際にすぐに気づくことができる

In [ ]:
# 0件チェック
#　テーブル
df.withColumn("count_check", F.when(F.lit(df.count()) > 0, F.lit(1))).show()

# カラム数チェック
スキーマが急に変更されていないかをチェックする

In [ ]:
# 今回は8カラムある
#　テーブル
df.withColumn("column_num_check", F.when(F.lit(len(df.columns)) == 8,F.lit(1))).show()

# データのリペア

データの不備を見つけたら、そのデータを修正したり削除したりする必要があります。

データリペアとしては再集計を行う方法があります  
再集計を行わずにできる方法もあり  
Update文が打てる場合もたまにありますが、APIの上限があったりと使いやすものはあまりありません。  
またDelete文がないので、削除するということはできません。

そうなると結局再集計という道に落ち着くことが多いです。  

今回は不要なデータを除いて再度利用するという再集計の方式を行ってみたいと思います。

途中で見つかったいらないデータ

```
|都道府県コード| 元号|和暦（年）|西暦（年）|  注|人口（総数）|人口（男）|  人口（女）|            true|
|1)　沖縄県は調査されなかったため...| null|  null|   null|null|    null|      null|        null|
|2)　長野県西筑摩群山口村と岐阜県...| null|  null|   null|null|    null|      null|        null|
```

In [ ]:
# データのリペア
df.filter(F.col("code")!="都道府県コード") \
    .filter(~F.col("code").contains("1)　沖縄県は調査され"))\
    .filter(~F.col("code").contains("2)　長野県西筑摩群山口村")).show()

# 軽く確認
df.count()
df.filter(F.col("code")!="都道府県コード") \
    .filter(~F.col("code").contains("1)　沖縄県は調査され"))\
    .filter(~F.col("code").contains("2)　長野県西筑摩群山口村")).count()

rep_df = df.filter(F.col("code")!="都道府県コード") \
    .filter(~F.col("code").contains("1)　沖縄県は調査され"))\
    .filter(~F.col("code").contains("2)　長野県西筑摩群山口村"))

rep_df.select(F.countDistinct("code","gengo", "wareki")/ rep_df.count()).show()

rep_df.withColumn("unique_ness_check", 
    F.lit(rep_df.agg(F.countDistinct("code","gengo", "wareki").alias("countdistinct")).collect()[0][0]) / F.lit(rep_df.count())).show()

# ストリーミングのテスト
ストリーミングのテストは、ストリーミングで流れている際に行われることはあまりありません。

その代わりストリーミングではテストが不要になるようにAvroというフォーマットでデータの担保されることが多いです。

## Avro フォーマットとは？

もう一つがHadoopの生みの親であるDoug Cutting氏によりプロジェクト化されたAvro（アブロ）フォーマットです【URL】https://avro.apache.org。  
Avroフォーマットはおもにストリーミングでのやり取りで効力を発揮するフォーマットです。 

元々AvroはHadoopの弱点であったJavaでしか読み書きできないという言語のポータビリティを解決するために生まれました言語の  
ポータビリティーが低いということはそのままAvroファイルと連携する対向のシステムの利用言語まで縛ってしまう可能性があります。 　

Avroフォーマットには、送信側に対する型や入力値を規定ことができます。


```
{
     "type": "record",
     "namespace": "com.example",
     "name": "FullName",
     "fields": [
       { "name": "first", "type": "string" },
       { "name": "last", "type": "string" }
     ]
} 
```

# じゃあテストしない？
それではテストしないか？というわけではなくて、ストリーミングもデータが溜まってくると結局データとしてはバッチデータとなります。

そのため、最終的には今まで見てきたようなテスト方法を適用することとなります。


In [ ]:
# 最後はSparkをクローズする
spark.stop()
spark.sparkContext.stop()